<a href="https://colab.research.google.com/github/viniciusrpb/cic0269_natural_language_processing/blob/main/lectures/cap10_2_metricas_concordancia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capítulo 10 - Construção e Anotação de Corpos de Textos


Fontes:

[Sklearn Cohen's Kappa](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html)

[Fast Krippendorff](https://github.com/pln-fing-udelar/fast-krippendorff)


## 10.2 - Métricas de Concordâncias entre Anotadores

In [ ]:
!pip install krippendorff

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import krippendorff
import numpy as np
import pandas as pd
from sklearn.metrics import cohen_kappa_score

Veja o seguinte exemplo (colocamos em um objeto DataFrame apenas para facilitar o entendimento):

In [ ]:
corpus = []

corpus.append(["O ser humano fede!","negativo","negativo","negativo"])
corpus.append(["Onde existem flores, existe vida!","positivo","positivo","positivo"])
corpus.append(["Choveu ontem lá no ICC","neutro","neutro","negativo"])
corpus.append(["Minha sogra me visitou ontem","negativo","negativo","neutro"])
corpus.append(["Agora são 13 horas e 14 minutos","positivo","neutro","negativo"])

df = pd.DataFrame(corpus, columns = ['texto','Label - Anotador 1','Label - Anotador 2','Label - Anotador 3'])

In [ ]:
df

,texto,Label - Anotador 1,Label - Anotador 2,Label - Anotador 3
0,O ser humano fede!,negativo,negativo,negativo
1,"Onde existem flores, existe vida!",positivo,positivo,positivo
2,Choveu ontem lá no ICC,neutro,neutro,negativo
3,Minha sogra me visitou ontem,negativo,negativo,neutro
4,Agora são 13 horas e 14 minutos,positivo,neutro,negativo


Alterando a representação do DataFrame para o formato de entrada de uma tabela em que:


*   Cada linha é um anotador
*   Cada coluna é um documento
*   Cada célula da tabela é o rótulo (\textit{label}) $v_{ij}$ atribuído pelo anotador $i$ para o documento $j$


    [
        {unit1:value, unit2:value, ...},  # coder 1
        {unit1:value, unit3:value, ...},   # coder 2
        ...                            # more coders
    ]



In [ ]:
corpus_transpose = []

for atrib in df:

    if atrib != 'texto':

        lista = np.array(df[atrib])
        corpus_transpose.append(list(lista.T))

Como resultado, obtemos uma matriz em que cada linha se refere a um anotador e as colunas os labels atribuídos para cada documento:

In [ ]:
corpus_transpose

[['negativo', 'positivo', 'neutro', 'negativo', 'positivo'],
 ['negativo', 'positivo', 'neutro', 'negativo', 'neutro'],
 ['negativo', 'positivo', 'negativo', 'neutro', 'negativo']]

Podemos também obter a representação inteira dos labels ao invés de strings:

In [ ]:
mapping = {'positivo' : 2, 'neutro' : 1, 'negativo' : 0}

In [ ]:
corpus_transpose_int = []

for labels in corpus_transpose:
    t = []
    for doc in labels:
        t.append(mapping[doc])
    corpus_transpose_int.append(t)

corpus_transpose_int

[[0, 2, 1, 0, 2], [0, 2, 1, 0, 1], [0, 2, 0, 1, 0]]

### Coeficiente de Cohen's Kappa $\kappa$

Lembre-se que o coeficiente Cohen's Kappa calcula a concordância entre dois anotadores apenas:

In [ ]:
print(cohen_kappa_score(corpus_transpose[0], corpus_transpose[1]))

0.7058823529411765


### Coeficiente de Krippendorff



Exemplo da documentação: com valores ausentes (*), isto é, o anotador (por algum motivo) não atribui rótulos

In [ ]:
reliability_data_str = ["*    *    *    *    *    3    4    1    2    1    1    3    3    *    3",  # anotador A
                        "1    *    2    1    3    3    4    3    *    *    *    *    *    *    *",  # anotador B
                        "*    *    2    1    3    4    4    *    2    1    1    3    3    *    4"]  # anotador C
print("\n".join(reliability_data_str))

*    *    *    *    *    3    4    1    2    1    1    3    3    *    3
1    *    2    1    3    3    4    3    *    *    *    *    *    *    *
*    *    2    1    3    4    4    *    2    1    1    3    3    *    4


In [ ]:
reliability_data = [[np.nan if v == "*" else int(v) for v in coder.split()] for coder in reliability_data_str]
reliability_data

[[nan, nan, nan, nan, nan, 3, 4, 1, 2, 1, 1, 3, 3, nan, 3],
 [1, nan, 2, 1, 3, 3, 4, 3, nan, nan, nan, nan, nan, nan, nan],
 [nan, nan, 2, 1, 3, 4, 4, nan, 2, 1, 1, 3, 3, nan, 4]]

Agora com matriz de contagem

In [ ]:
print("Krippendorff's alpha for nominal metric: ", krippendorff.alpha(reliability_data=reliability_data,
                                                                          level_of_measurement="nominal"))

print("From value counts:\n")

value_counts = np.array([[1, 0, 0, 0],
                         [0, 0, 0, 0],
                         [0, 2, 0, 0],
                         [2, 0, 0, 0],
                         [0, 0, 2, 0],
                         [0, 0, 2, 1],
                         [0, 0, 0, 3],
                         [1, 0, 1, 0],
                         [0, 2, 0, 0],
                         [2, 0, 0, 0],
                         [2, 0, 0, 0],
                         [0, 0, 2, 0],
                         [0, 0, 2, 0],
                         [0, 0, 0, 0],
                         [0, 0, 1, 1]])
print(value_counts)
print("Krippendorff's alpha for nominal metric: ", krippendorff.alpha(value_counts=value_counts,
                                                                          level_of_measurement="nominal"))

Krippendorff's alpha for nominal metric:  0.691358024691358
From value counts:

[[1 0 0 0]
 [0 0 0 0]
 [0 2 0 0]
 [2 0 0 0]
 [0 0 2 0]
 [0 0 2 1]
 [0 0 0 3]
 [1 0 1 0]
 [0 2 0 0]
 [2 0 0 0]
 [2 0 0 0]
 [0 0 2 0]
 [0 0 2 0]
 [0 0 0 0]
 [0 0 1 1]]
Krippendorff's alpha for nominal metric:  0.691358024691358


## 10.3 - Avaliação por Similaridade entre Textos

In [ ]:
texto_anotador_a = "uma parte de um texto"
texto_anotador_b = "uma parte de um texto"
texto_anotador_c = "uma parte de um texto que vai longe"
texto_anotador_d = "Visando obter uma parte de um texto que vai longe"

Edit Distance

Coeficiente de Jaccard

SyntaxError: ignored